In [ ]:
!pip install wordcloud
!pip install sklearn
!pip install tensorflow
import re
import nltk
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split,KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
emotions_dict = {
    "num":{ 0: "sadness",
        1: "anger",
        2: "love",
        3: "surprise",
        4: "fear",
        5: "joy"}
}

tag_id_dict = {}
for key, value in emotions_dict["num"].items():
    tag_id_dict[value] = key

emotions_dict["num"] = tag_id_dict

In [ ]:
train = pd.read_csv('C:/Users/NEGIN COMPUTER/Downloads/archive/train.txt', names=['Text', 'label'], delimiter=';')
val = pd.read_csv('C:/Users/NEGIN COMPUTER/Downloads/archive/val.txt', names=['Text', 'label'], delimiter=';')
test = pd.read_csv('C:/Users/NEGIN COMPUTER/Downloads/archive/test.txt', names=['Text', 'label'], delimiter=';')

In [ ]:
val.head()

In [ ]:
val.shape

In [ ]:
train.label.value_counts()

In [ ]:
train.isnull().sum()

In [ ]:
import seaborn as sns
sns.countplot(train.label)

In [ ]:
train.duplicated().sum()

In [ ]:
index = train[train.duplicated() == True].index
train.drop(index, axis = 0, inplace = True)
train.reset_index(inplace=True, drop = True)

In [ ]:
index = train[train['Text'].duplicated() == True].index
train.drop(index, axis = 0, inplace = True)
train.reset_index(inplace=True, drop = True)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_data(train, test, val, max_length=None):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train)

    vocab_size = len(tokenizer.word_index) + 1
    print(f"Vocabulary size: {vocab_size}")

    if max_length is None:
        max_length = max(len(sequence.split()) for sequence in train)

    train_sequences = tokenizer.texts_to_sequences(train)
    test_sequences = tokenizer.texts_to_sequences(test)
    val_sequences = tokenizer.texts_to_sequences(val)

    x_train = pad_sequences(train_sequences, maxlen=max_length, padding='post')
    x_test = pad_sequences(test_sequences, maxlen=max_length, padding='post')
    x_val = pad_sequences(val_sequences, maxlen=max_length, padding='post')

    print("x_train shape:", x_train.shape)
    print("x_test shape:", x_test.shape)
    print("x_val shape:", x_val.shape)

    return x_train, x_test, x_val, vocab_size, tokenizer

In [ ]:
x_train, x_test, x_val, VOCAB_SIZE, tokenizer = data_preprocessing(train, test, val)

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_preprocessing(data):
    encoder = LabelEncoder()
    y = encoder.fit_transform(data["label"].to_list())
    y = y.reshape(-1, 1)
    return y

y_train = label_preprocessing(train)
y_test = label_preprocessing(test)
y_val = label_preprocessing(val)

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
labels = ['sadness', 'anger', 'love', 'surprise', 'fear','joy']
encoder.fit(labels)

VOCAB_SIZE = 1000
embedding_dim = 64
hidden_units = 10
num_labels = len(encoder.classes_)

inputs = tf.keras.Input(shape=(30,))

x = tf.keras.layers.Embedding(VOCAB_SIZE, embedding_dim, input_length=30)(inputs)

x = tf.keras.layers.LSTM(hidden_units, return_sequences=True)(x)

x = tf.keras.layers.Flatten()(x)

output = tf.keras.layers.Dense(num_labels, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=output)

model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])

In [ ]:
history = model.fit(x_train, y_train, epochs=20, batch_size=32,
                    validation_data=(x_val, y_val))

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
class Predict:
    def __init__(self, model, tokenizer, tokenizer):
        self.model = model
        self.tokenizer = Tokenizer()
    
    def predict(self, txt):
        x = self.tokenizer.texts_to_sequences([txt])
        x = tf.keras.preprocessing.sequence.pad_sequences(x, maxlen=30)
        predictions = self.model.predict(x)
        predicted_label_index = np.argmax(predictions[0])
        predicted_label = self.emotions_dict["num"][predicted_label_index]
        return predicted_label

predict = Predict(model, tokenizer, emotions_dict)

In [ ]:
predict = Predict(model, tokenizer)
predict.predict("im so sad!")